In [1]:
import pandas as pd
import os
from datetime import date, timedelta
from sqlalchemy import create_engine, text

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}
pd.set_option('display.float_format','{:,.2f}'.format)
year = 2025
refund = 57_000   # 66,490.89 Windfall = 200,000 March
                    # LTF = 181498.8 + 3396.97 + 50380.22 = 235,000 June
                    # 501_000 + 167_000 = 668_000
dividend = 0 #361_000
expense  = 44_000

today = date.today()
today

datetime.date(2025, 2, 3)

### Restart and Run All Cells

In [32]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [ ]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

### Record selection for transactions

In [4]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % year
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2025
ORDER BY sells.date DESC, name


In [5]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,net,kind


In [6]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum()

Series([], Name: net, dtype: object)

In [7]:
sells_df.groupby(['name'])[['gross','net']].sum().head(5)

,gross,net
name,,


In [8]:
#total_df = sells_df.groupby(['name'])[['gross','net']].sum()
#total_df.nlargest(5, 'gross')[['gross','net']].style.format(format_dict)

TypeError: Column 'gross' has dtype object, cannot use method 'nlargest' with this dtype

In [34]:
file_name = 'ytd-profit.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [36]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\ytd-profit.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\ytd-profit.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\ytd-profit.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\ytd-profit.csv


In [38]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(output_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(god_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(icd_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(osd_file)

In [40]:
output = sells_df.groupby(['sell_month']).net.sum()
output

Series([], Name: net, dtype: object)

In [42]:
file_name = 'tmp-file-of-profits.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [46]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}")

Output file : C:\Users\PC1\OneDrive\A5\Data\tmp-file-of-profits.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\tmp-file-of-profits.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\tmp-file-of-profits.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\tmp-file-of-profits.csv


In [48]:
output.to_csv(output_file, index=True)
output.to_csv(god_file, index=True)
output.to_csv(icd_file, index=True)
output.to_csv(osd_file, index=True)

In [50]:
profit = sells_df.net.sum()
ttl_profit = profit + dividend
profit,dividend,ttl_profit

(0, 0, 0)

In [52]:
profit = sells_df.net.sum()
net = profit + dividend - expense
net = round(net / 1000,0) * 1000
round(profit,0),dividend,expense, round(net,0)

(0, 0, 44000, -44000.0)

### Bottom line of up to date transactions

In [55]:
trade = round(profit,0) + dividend
trade = round(trade / 1000,0) * 1000
trade

0.0

In [57]:
total = refund + net
refund, round(net,0), round(total,0)

(57000, -44000.0, 13000.0)